# Do the scoliosis EMG analysis for one patient

In [ ]:
import pathlib

import numpy as np
from collections import defaultdict

import pandas as pd

from gaitutils.stats import collect_trial_data
from gaitutils.trial import Trial
from gaitutils import eclipse
from gaitutils.config import cfg

import matplotlib.pyplot as plt
#%matplotlib widget

In [ ]:
DATA_FLDR = 'Z:/Skolioosi/Potilaina/E0125_AK'
WALKING_TAGS = {'E1', 'E2', 'E3', 'E4', 'T1', 'T2', 'T3', 'T4'}
RUNNING_TAGS = {'J1', 'J2', 'J3', 'J4', 'J5', 'J6'}
MVC_TAG = 'MVC'

REMOVE_MARG = 2 # seconds, only applies to MVC
CONV_KERN_LENGTH = 2 # seconds
cfg.trial.no_toeoff = 'reject'
cfg.emg.envelope_method = 'linear_envelope'
cfg.emg.linear_envelope_lowpass = 10

## Read all the walking files

In [ ]:
emg_walk = defaultdict(lambda: np.zeros((1000,0)))

for c3d_file in pathlib.Path(DATA_FLDR).glob('*.c3d'):
    print(f'Reading file {c3d_file} ...')
    trial = Trial(c3d_file)
    
    if trial.eclipse_tag in WALKING_TAGS:
        print(f'Collecting trial data for file {c3d_file} (walking) ...')
        data, cycles = collect_trial_data(trial, analog_envelope=True, force_collect_all_cycles=True, fp_cycles_only=False)
        for ch in data['emg'].keys():
            emg_walk[ch] = np.hstack((emg_walk[ch], data['emg'][ch].T))

## Read all the running files

In [ ]:
emg_run = defaultdict(lambda: np.zeros((1000,0)))

for c3d_file in pathlib.Path(DATA_FLDR).glob('*.c3d'):
    print(f'Reading file {c3d_file} ...')
    trial = Trial(c3d_file)
    
    if any(tag in eclipse.get_eclipse_keys(trial.enfpath)['NOTES'] for tag in RUNNING_TAGS) or any(tag in eclipse.get_eclipse_keys(trial.enfpath)['DESCRIPTION'] for tag in RUNNING_TAGS):
        print(f'Collecting trial data for file {c3d_file} (running) ...')
        data, cycles = collect_trial_data(trial, analog_envelope=True, force_collect_all_cycles=True, fp_cycles_only=False)
        for ch in data['emg'].keys():
            emg_run[ch] = np.hstack((emg_run[ch], data['emg'][ch].T))

## Read all the reference files

In [ ]:
emg_mvc = defaultdict(lambda: np.zeros((0,)))
sfrate = None

for c3d_file in pathlib.Path(DATA_FLDR).glob('*.c3d'):
    trial = Trial(c3d_file)
    trial.get_emg_data(next(iter(emg_walk.keys())))

    if sfrate is None:
        sfrate = trial.emg.sfrate
    else:
        # All the trials should have the same sampling rate
        assert sfrate == trial.emg.sfrate


    if MVC_TAG in eclipse.get_eclipse_keys(trial.enfpath)['NOTES'] or MVC_TAG in eclipse.get_eclipse_keys(trial.enfpath)['DESCRIPTION']:
        print(f'Reading file {c3d_file} (MVC) ...')
        for ch in emg_walk.keys():
            ch_data = trial.get_emg_data(ch, envelope=True)[1]
            ch_data = ch_data[round(REMOVE_MARG*sfrate):-round(REMOVE_MARG*sfrate)]
            emg_mvc[ch] = np.concatenate((emg_mvc[ch], ch_data))

## Find maxima

In [ ]:
kernel = np.ones(round(CONV_KERN_LENGTH * sfrate))    # Note that we are using sampling rate from the last trial we read, assuming that all trials have the same sampling rate
kernel /= kernel.sum()

emg_mvc_max = {}

for ch in emg_walk:
    emg_mvc_max[ch] = np.convolve(emg_mvc[ch], kernel, mode='valid').max()

## Normalize

In [ ]:
for ch in emg_walk:
    emg_walk[ch] /= emg_mvc_max[ch]
    emg_run[ch] /= emg_mvc_max[ch]

## Plot (walking)

In [ ]:
ch_base = set(ch[1:] for ch in emg_walk.keys())

for ch in ch_base:
    plt.figure()
    
    plt.subplot(1, 2, 1)
    plt.plot(emg_walk[f'R{ch}'], color='green')
    plt.plot(emg_walk[f'L{ch}'], color='red')
    plt.title(f'{ch}')
    
    plt.subplot(1, 2, 2)
    plt.plot(emg_walk[f'R{ch}'].mean(axis=1), color='green')
    plt.plot(emg_walk[f'R{ch}'].mean(axis=1) + emg_walk[f'R{ch}'].std(axis=1), color='green', linestyle='dotted')
    plt.plot(emg_walk[f'R{ch}'].mean(axis=1) - emg_walk[f'R{ch}'].std(axis=1), color='green', linestyle='dotted')

    plt.plot(emg_walk[f'L{ch}'].mean(axis=1), color='red')
    plt.plot(emg_walk[f'L{ch}'].mean(axis=1) + emg_walk[f'L{ch}'].std(axis=1), color='red', linestyle='dotted')
    plt.plot(emg_walk[f'L{ch}'].mean(axis=1) - emg_walk[f'L{ch}'].std(axis=1), color='red', linestyle='dotted')

    plt.title(f'{ch} (avg + std)')
    plt.gcf().set_size_inches(20, 6)


## Compute the indices (walk)

In [ ]:
res = {'index': ('right max', 'std of (right max)', 'left max', 'std of (left max)', 'right average', 'left average', 'right mvc', 'left mvc')}
                 
for ch in ch_base:
    res[ch] = (np.mean(emg_walk[f'R{ch}'].max(axis=0)),
               np.std(emg_walk[f'R{ch}'].max(axis=0)),
               np.mean(emg_walk[f'L{ch}'].max(axis=0)),
               np.std(emg_walk[f'L{ch}'].max(axis=0)),
               emg_walk[f'R{ch}'].mean(),
               emg_walk[f'L{ch}'].mean(),
               emg_mvc_max[f'R{ch}'],
               emg_mvc_max[f'L{ch}'])
    
df = pd.DataFrame(res)
df

## Plot (running)

In [ ]:
ch_base = set(ch[1:] for ch in emg_run.keys())

for ch in ch_base:
    plt.figure()
    
    plt.subplot(1, 2, 1)
    plt.plot(emg_run[f'R{ch}'], color='green')
    plt.plot(emg_run[f'L{ch}'], color='red')
    plt.title(f'{ch}')
    
    plt.subplot(1, 2, 2)
    plt.plot(emg_run[f'R{ch}'].mean(axis=1), color='green')
    plt.plot(emg_run[f'R{ch}'].mean(axis=1) + emg_run[f'R{ch}'].std(axis=1), color='green', linestyle='dotted')
    plt.plot(emg_run[f'R{ch}'].mean(axis=1) - emg_run[f'R{ch}'].std(axis=1), color='green', linestyle='dotted')

    plt.plot(emg_run[f'L{ch}'].mean(axis=1), color='red')
    plt.plot(emg_run[f'L{ch}'].mean(axis=1) + emg_run[f'L{ch}'].std(axis=1), color='red', linestyle='dotted')
    plt.plot(emg_run[f'L{ch}'].mean(axis=1) - emg_run[f'L{ch}'].std(axis=1), color='red', linestyle='dotted')

    plt.title(f'{ch} (avg + std)')
    plt.gcf().set_size_inches(20, 6)

## Compute the indices (run)

In [ ]:
res = {'index': ('right max', 'std of (right max)', 'left max', 'std of (left max)', 'right average', 'left average', 'right mvc', 'left mvc')}
                 
for ch in ch_base:
    res[ch] = (np.mean(emg_run[f'R{ch}'].max(axis=0)),
               np.std(emg_run[f'R{ch}'].max(axis=0)),
               np.mean(emg_run[f'L{ch}'].max(axis=0)),
               np.std(emg_run[f'L{ch}'].max(axis=0)),
               emg_run[f'R{ch}'].mean(),
               emg_run[f'L{ch}'].mean(),
               emg_mvc_max[f'R{ch}'],
               emg_mvc_max[f'L{ch}'])
    
df = pd.DataFrame(res)
df

## Scratch area